In [ ]:
import torch
import torchvision.datasets
import torchvision.transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math

class CryptoNet(nn.Module):
    def __init__(self):
        super(CryptoNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(5)
        self.conv2 = nn.Conv2d(5, 10, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(10)
        self.fc1 = nn.Linear(2560, 10, bias =  False)
        self.fc2 = nn.Linear(10, 10, bias = False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu (x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        return x


model = CryptoNet()
network_state_dict = torch.load('padded_cifar_model.pth')
model.load_state_dict(network_state_dict)

print(model.fc1.weight.t().shape)
print(model.fc2.weight.t().shape)

model.eval()

torch.Size([4096, 10])
torch.Size([10, 10])


CryptoNet(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4096, out_features=10, bias=False)
  (fc2): Linear(in_features=10, out_features=10, bias=False)
)

In [2]:
test_dataset = torch.utils.data.DataLoader(
  torchvision.datasets.CIFAR10(root='imgs/', train=False,
                               download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                             ])),
  batch_size=32, shuffle=True)

## Initial Layer

In [3]:
values = []

for img, label in test_dataset:
    v = model.bn1(model.conv1(img)).reshape(-1).detach().numpy()
    values.append(max(v))
    values.append(min(v))
    
print("Min: {}\nMax: {}\nSuggested Delta: {}".format(min(values), max(values), 1 / (max(abs(min(values)), abs(max(values))))))

Min: -1.9983596801757812
Max: 1.5004990100860596
Suggested Delta: 0.5004104375839233


In [4]:
values = []

for img, label in test_dataset:
    v = F.relu(model.bn1(model.conv1(img)))
    v = model.bn2(model.conv2(v)).reshape(-1).detach().numpy()
    values.append(max(v))
    values.append(min(v))
    
print("Min: {}\nMax: {}\nSuggested Delta: {}".format(min(values), max(values), 1 / (max(abs(min(values)), abs(max(values))))))

Min: -3.479012966156006
Max: 3.516836643218994
Suggested Delta: 0.28434643149375916


In [5]:
values = []

for img, label in test_dataset:
    v = F.relu(model.bn1(model.conv1(img)))
    v = F.relu(model.bn2(model.conv2(v)))
    v = v.view(v.size(0), -1)
    v = model.fc1(v).reshape(-1).detach().numpy()
    values.append(max(v))
    values.append(min(v))
    
print("Min: {}\nMax: {}\nSuggested Delta: {}".format(min(values), max(values), 1 / (max(abs(min(values)), abs(max(values))))))

Min: -7.464317798614502
Max: 52.481082916259766
Suggested Delta: 0.019054485484957695
